In [4]:
import fitz  # PyMuPDF
import json
import logging
import time
from pathlib import Path
import pandas as pd
import io
from PIL import Image
import cv2
import numpy as np
import re
import unicodedata
from collections import defaultdict
from rapidfuzz import fuzz

In [5]:
JSON_INPUT = "chunks (1).json"  # your input JSON containing MANY chunks
PDF_FOLDER = ""  # folder where PDFs are located
JSON_OUTPUT = "bboxes_output.json"  # output file with bbox results
PDF_PATH = "2312.00752v2.pdf"
OUTPUT_JSON = "page1_words.json"
# -----------------------------------------------

In [6]:
OUTPUT = "chunks.json"

with open(JSON_INPUT, "r", encoding="utf-8") as f:
    data = json.load(f)

with open(OUTPUT, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Saved pretty JSON → {OUTPUT}")

Saved pretty JSON → chunks.json


In [7]:
OUTPUT_WORDS = "page1_words.json"
OUTPUT_BLOCKS = "page1_blocks.json"
OUTPUT_DICT = "page1_dict.json"
OUTPUT_RAW = "page1_raw.json"
OUTPUT_SPANS = "page1_spans.json"

doc = fitz.open(PDF_PATH)
page = doc[5]  # first page

# -------------------------------------------------------------------
# 1. WORDS
# -------------------------------------------------------------------
words = page.get_text("words")

words_formatted = []
for w in words:
    words_formatted.append(
        {
            "x0": w[0],
            "y0": w[1],
            "x1": w[2],
            "y1": w[3],
            "text": w[4],
            "block": w[5],
            "line": w[6],
            "word_index": w[7],
        }
    )

with open(OUTPUT_WORDS, "w", encoding="utf-8") as f:
    json.dump(words_formatted, f, indent=2, ensure_ascii=False)

print(f"[OK] Saved → {OUTPUT_WORDS}")

# -------------------------------------------------------------------
# 2. BLOCKS (block-level layout)
# -------------------------------------------------------------------
blocks = page.get_text("blocks")  # list of blocks
# blocks structure: (x0, y0, x1, y1, "block text", block_type, block_no)

blocks_formatted = []
for b in blocks:
    blocks_formatted.append(
        {
            "x0": b[0],
            "y0": b[1],
            "x1": b[2],
            "y1": b[3],
            "text": b[4],
            "type": b[5],  # 0 = text, 1 = image, etc.
            "block_no": b[6],
        }
    )

with open(OUTPUT_BLOCKS, "w", encoding="utf-8") as f:
    json.dump(blocks_formatted, f, indent=2, ensure_ascii=False)

print(f"[OK] Saved → {OUTPUT_BLOCKS}")

# -------------------------------------------------------------------
# 3. FULL PAGE DICT (block → lines → spans → glyphs)
# -------------------------------------------------------------------
page_dict = page.get_text("dict")

with open(OUTPUT_DICT, "w", encoding="utf-8") as f:
    json.dump(page_dict, f, indent=2, ensure_ascii=False)

print(f"[OK] Saved → {OUTPUT_DICT}")

# -------------------------------------------------------------------
# 4. RAW PyMuPDF JSON (EXACT raw with no processing)
# -------------------------------------------------------------------
raw = page.get_text("rawdict")  # preserves numeric formatting

with open(OUTPUT_RAW, "w", encoding="utf-8") as f:
    json.dump(raw, f, indent=2, ensure_ascii=False)

print(f"[OK] Saved → {OUTPUT_RAW}")


def get_spans_from_page(page):
    spans_formatted = []

    page_dict = page.get_text("dict")

    global_span_index = 0  # <--- ADD THIS

    for b_idx, block in enumerate(page_dict.get("blocks", [])):
        if block["type"] != 0:
            continue  # skip non-text blocks

        for l_idx, line in enumerate(block.get("lines", [])):
            line_bbox = line["bbox"]

            for s_idx, span in enumerate(line.get("spans", [])):
                spans_formatted.append(
                    {
                        "global_span_index": global_span_index,  # <--- ADD THIS
                        "block_index": b_idx,
                        "line_index": l_idx,
                        "span_index": s_idx,
                        "text": span["text"],
                        "span_bbox": span["bbox"],
                        "line_bbox": line_bbox,
                        "size": span.get("size"),
                        "font": span.get("font"),
                        "color": span.get("color"),
                        "flags": span.get("flags"),
                    }
                )

                global_span_index += 1

    with open(OUTPUT_SPANS, "w", encoding="utf-8") as f:
        json.dump(spans_formatted, f, indent=2, ensure_ascii=False)

    print(f"[OK] Saved → {OUTPUT_SPANS}")


doc.close()

[OK] Saved → page1_words.json
[OK] Saved → page1_blocks.json
[OK] Saved → page1_dict.json
[OK] Saved → page1_raw.json


In [8]:
def extract_spans(page):
    dict_data = page.get_text("dict")
    spans = []

    for block in dict_data.get("blocks", []):
        if block["type"] != 0:
            continue  # only text blocks

        for line in block.get("lines", []):
            for span in line.get("spans", []):
                spans.append(
                    {
                        "text": span["text"],
                        "bbox": span["bbox"],
                        "line_bbox": line["bbox"],
                        "font": span["font"],
                        "size": span["size"],
                    }
                )
    return spans

In [9]:
import ftfy
from unidecode import unidecode
import unicodedata
import re


# -----------------------------------------------------------
# 1. Convert Greek / Math Unicode → ASCII semantic names
#    e.g. Δ → delta, 𝛥 → delta, 𝛼 → alpha, ℎ → h, 𝑐 → c
# -----------------------------------------------------------
def fold_math_and_greek(text: str) -> str:
    out = []

    for ch in text:
        try:
            name = unicodedata.name(ch)
        except ValueError:
            # unprintable or private unicode
            continue

        # A) Greek letters (all variants)
        if "GREEK" in name:
            # Example names:
            # "GREEK CAPITAL LETTER DELTA"
            # "GREEK SMALL LETTER ALPHA"
            #
            # We extract the final word:
            base = name.split("LETTER")[-1].strip().lower()
            out.append(base)
            continue

        # B) Mathematical alphanumeric symbols (𝛥, 𝑥, 𝒶, 𝔸, 𝕒)
        if "MATHEMATICAL" in name:
            # Example names:
            # "MATHEMATICAL BOLD CAPITAL DELTA"
            # "MATHEMATICAL ITALIC SMALL X"
            parts = name.split()
            # last part is usually the base letter ("DELTA", "X", etc.)
            base = parts[-1].lower()
            out.append(base)
            continue

        # C) All other characters: keep raw
        out.append(ch)

    return "".join(out)


# -----------------------------------------------------------
# 2. Main Normalization (robust)
# -----------------------------------------------------------
def normalize(text: str) -> str:
    if not text:
        return ""

    # Step 1: Fix broken Unicode encodings
    text = ftfy.fix_text(text)

    # Step 2: Fold Greek & Math Unicode to ASCII words
    text = fold_math_and_greek(text)

    # Step 3: Convert fancy unicode → closest ASCII
    text = unidecode(text)

    # Step 4: Lowercase
    text = text.lower()

    # Step 5: Remove stray symbols except basic punctuation
    text = re.sub(r"[^a-z0-9\s.,;:()\[\]{}+\-=/]", " ", text)

    # Step 6: Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text


from difflib import SequenceMatcher


def fuzzy(a, b):
    return SequenceMatcher(None, a, b).ratio()


def match_percentage(combined, chunk):
    """
    Percentage of chunk that appears in combined.
    No penalty for unmatched prefix/suffix.
    """
    if not chunk:
        return 0.0
    if not combined:
        return 0.0

    matcher = SequenceMatcher(None, chunk, combined)

    matched = 0
    for block in matcher.get_matching_blocks():
        matched += block.size  # number of matching chars

    pct = (matched / len(chunk)) * 100
    return round(pct, 2)

In [10]:
def get_seed(chunk_text, lengths=[10, 5, 2]):
    words = normalize(chunk_text).split()
    return ["".join(words[:l]) for l in lengths if len(words) >= l]


def find_candidate_spans(spans, seeds, seed_threshold=0.40):
    matched_indices = set()
    for seed in seeds:
        for idx, sp in enumerate(spans):
            score = fuzzy(normalize(sp["text"]), seed)
            if score >= seed_threshold:
                matched_indices.add(idx)
    print(f"Found {len(matched_indices)} unique candidate spans across seeds.")
    return list(matched_indices)


def expand_best_span_match(spans, chunk_text, candidate_indices, window=8):
    normalized_chunk = normalize(chunk_text)

    best_score = 0
    best_range = None

    best_combined = ""
    for start in candidate_indices:

        combined = ""

        for j in range(start, min(start + window, len(spans))):

            combined += "" + spans[j]["text"]
            score = match_percentage(normalize(combined), normalized_chunk)

            if score > best_score:
                best_score = score

                best_range = (start, j)
                best_combined = combined

    print(f"Best match score: {best_score:.4f} for spans {best_range}")
    print(f"Best matched text: '{best_combined}'")
    return best_range, best_score


def find_best_span_match(spans, chunk_text, threshold=0.80):
    seed = get_seed(chunk_text)
    candidates = find_candidate_spans(spans, seed)

    if not candidates:
        return None, 0

    best_range, best_score = expand_best_span_match(
        spans, chunk_text, candidates, window=100  # look ahead only 10 spans
    )

    if best_score >= threshold:
        return best_range, best_score

    return None, best_score

In [11]:
def merge_spans_into_line_bboxes(spans, span_range):
    (start, end) = span_range
    selected = spans[start : end + 1]

    line_map = {}

    for sp in selected:
        lx0, ly0, lx1, ly1 = sp["line_bbox"]
        if (ly0, ly1) not in line_map:
            line_map[(ly0, ly1)] = [lx0, ly0, lx1, ly1]
        else:
            line_map[(ly0, ly1)][0] = min(line_map[(ly0, ly1)][0], lx0)
            line_map[(ly0, ly1)][2] = max(line_map[(ly0, ly1)][2], lx1)

    # convert dict to list
    bboxes = []
    for k, v in line_map.items():
        bboxes.append({"x0": v[0], "y0": v[1], "x1": v[2], "y1": v[3]})

    # keep in vertical reading order
    bboxes.sort(key=lambda b: b["y0"])

    return bboxes


def normalize_bboxes(bboxes, page_width, page_height):
    normalized = []
    for box in bboxes:
        norm_box = {
            "x0": box["x0"] / page_width,
            "y0": box["y0"] / page_height,
            "x1": box["x1"] / page_width,
            "y1": box["y1"] / page_height,
        }
        normalized.append(norm_box)
    return normalized

In [12]:
def match_chunk_by_span(page, chunk_text, threshold=0.80):
    spans = extract_spans(page)
    span_range, score = find_best_span_match(spans, chunk_text, threshold)

    if not span_range:
        return {"found": False, "score": score, "bboxes": []}

    bboxes = merge_spans_into_line_bboxes(spans, span_range)

    # Get page size from PyMuPDF
    page_width, page_height = page.rect.width, page.rect.height

    normalized_bboxes = normalize_bboxes(bboxes, page_width, page_height)

    return {"found": True, "score": score, "bboxes": normalized_bboxes}

In [13]:
doc = fitz.open("2312.00752v2.pdf")
page = doc[5 - 1]
get_spans_from_page(page)
chunk = "To understand this principle, we focus on two running examples of synthetic tasks (Figure 2). • The Selective Copying task modifies the popular Copying task (Arjovsky, Shah, and Bengio 2016) by varying the position of the tokens to memorize. It requires content-aware reasoning to be able to memorize the relevant tokens (colored) and filter out the irrelevant ones (white). • The Induction Heads task is a well-known mechanism hypothesized to explain the majority of in-context learning abilities of LLMs (Olsson et al. 2022)."

result = match_chunk_by_span(page, chunk, threshold=0.75)

print(result)

[OK] Saved → page1_spans.json
Found 8 unique candidate spans across seeds.
Best match score: 99.4300 for spans (28, 51)
Best matched text: 'To understand this principle, we focus on two running examples of synthetic tasks (Figure 2).• The Selective Copying task modifies the popular Copying task (Arjovsky, Shah, and Bengio 2016) by varying theposition of the tokens to memorize. It requires content-aware reasoning to be able to memorize the relevant tokens(colored) and filter out the irrelevant ones (white).• The Induction Heads task is a well-known mechanism hypothesized to explain the majority of in-context learningabilities of LLMs (Olsson et al. 2022). It requires'
{'found': True, 'score': 99.43, 'bboxes': [{'x0': 0.11715849084791795, 'y0': 0.35183850683347145, 'x1': 0.737118590111826, 'y1': 0.3644175673976089}, {'x0': 0.11764705882352941, 'y0': 0.374356549195569, 'x1': 0.9176506092345792, 'y1': 0.39011051678898356}, {'x0': 0.13149999481400634, 'y0': 0.3894512677433515, 'x1': 0.91764

In [14]:
with open(JSON_INPUT, "r", encoding="utf-8") as f:
    items = json.load(f)

results = []

# --------------------------
# Group by document_key extracted from doc_id
# --------------------------

pdf_groups = {}
for item in items:
    doc_id = item["doc_id"]  # e.g. "2312.00752v2-merged-document"
    page = item["page"]  # page number
    text = item["text"]

    # Extract document_key before "-merged-document"
    if "-merged-document" in doc_id:
        document_key = doc_id.split("-merged-document")[0]
    else:
        document_key = doc_id

    entry = {
        "document_key": document_key,
        "page_number": page,
        "text": text,
    }

    pdf_groups.setdefault(document_key, []).append(entry)

# --------------------------
# Process each PDF
# --------------------------

for doc_key, entries in pdf_groups.items():

    pdf_path = Path(PDF_FOLDER) / f"{doc_key}.pdf"

    if not pdf_path.exists():
        print(f"[WARN] PDF missing → {pdf_path}")
        continue

    print(f"Processing PDF: {pdf_path}")

    doc = fitz.open(pdf_path)

    for ch in entries:
        page_num = ch["page_number"] - 1  # convert to zero-based index
        if page_num < 0 or page_num >= len(doc):
            print(f"[WARN] Invalid page {page_num+1} in PDF {doc_key}")
            continue

        page = doc[page_num]

        # --------------------------
        # NEW MATCHING FUNCTION
        # --------------------------
        bboxes = match_chunk_by_span(page, ch["text"], threshold=0.78)

        results.append(
            {
                "document_key": doc_key,
                "page_number": ch["page_number"],
                "text": ch["text"],
                "bboxes": bboxes,
            }
        )

    doc.close()

# --------------------------
# Save output
# --------------------------

with open(JSON_OUTPUT, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"Done. Saved bbox results → {JSON_OUTPUT}")

Processing PDF: 2312.00752v2.pdf
Found 4 unique candidate spans across seeds.
Best match score: 98.2700 for spans (0, 14)
Best matched text: 'Mamba: Linear-Time Sequence Modeling with Selective State SpacesAlbert Gu∗1 and Tri Dao∗21Machine Learning Department, Carnegie Mellon University2Department of Computer Science, Princeton Universityagu@cs.cmu.edu, tri@tridao.me'
Found 5 unique candidate spans across seeds.
Best match score: 99.1100 for spans (5, 33)
Best matched text: 'and Tri Dao∗21Machine Learning Department, Carnegie Mellon University2Department of Computer Science, Princeton Universityagu@cs.cmu.edu, tri@tridao.meAbstractFoundation models, now powering most of the exciting applications in deep learning, are almost universally based on theTransformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention,gated convolution and recurrent models, and structured state space models (SSMs) have been developed to addressTransformers’

In [15]:
doc = fitz.open("2312.00752v2.pdf")
page = doc[0]  # Get the first page

search_text = "Abstract Foundation models, now powering most of the exciting applications in deep learning, are almost universally based on the Transformer architecture and its core attention module. Many subquadratic-time architectures such as linear attention, gated convolution and recurrent models, and structured state space models (SSMs) have been developed to address Transformers’ computational inefficiency on long sequences, but they have not performed as well as attention on important modalities such as language. We identify that a key weakness of such models is their inability to perform content-based reasoning, and make several improvements. First, simply letting the SSM parameters be functions of the input addresses their weakness with discrete modalities, allowing the model to selectively propagate or forget information along the sequence length dimension depending on the current token. Second, even though this change prevents the use of efficient convolutions, we design a hardware-aware parallel algorithm in recurrent mode. We integrate these selective SSMs into a simplified end-to-end neural network architecture without attention or even MLP blocks ( Mamba ). Mamba enjoys fast inference (5 × higher throughput than Transformers) and linear scaling in sequence length, and its performance improves on real data up to million-length sequences."
text_instances = page.search_for(search_text)

for inst in text_instances:
    # 'inst' is a Rect object representing the bounding box
    print(f"Bounding Box: {inst}")
    print(f"Coordinates: x0={inst.x0}, y0={inst.y0}, x1={inst.x1}, y1={inst.y1}")

doc.close()